In [ ]:
import io
import numpy as np
import pandas as pd

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving power_issued_1.csv to power_issued_1.csv
Saving power_issued_2.csv to power_issued_2.csv
Saving power_issued_3.csv to power_issued_3.csv
Saving power_issued_4.csv to power_issued_4.csv
Saving power_issued_5.csv to power_issued_5.csv
Saving power_issued_6.csv to power_issued_6.csv
Saving power_issued_7.csv to power_issued_7.csv
Saving power_issued_8.csv to power_issued_8.csv
Saving power_issued_9.csv to power_issued_9.csv
Saving power_issued_10.csv to power_issued_10.csv


# Práctica 1 - Las leyes de la probabilidad

## Ejercicio 1 - Las mamografías como prueba diagnóstica en tumores de mama

### Prevalencia del cáncer de mama

En el fichero de mammogram.csv se encuentran los resultados de una serie de mamografías hechas a un grupo de mujeres adultas de todas las edades. Estas mujeres se hicieron dos mamografías sucesivas y, algunas de ellas, fueron posteriormente diagnosticadas con lesiones malignas.

Según los datos del estudio, ¿cuál es la probabilidad empírica de padecer una lesión maligna de mama?

In [ ]:
path = io.BytesIO(uploaded['mammogram.csv'])
df = pd.read_csv(path, sep=",")

df

,Patient,hadCancer,mammoResult1,mammoResult2
0,1,0,0,0
1,2,0,0,1
2,3,0,0,0
3,4,1,1,1
4,5,0,0,0
...,...,...,...,...
199995,199996,0,0,0
199996,199997,1,1,1
199997,199998,0,1,0
199998,199999,0,0,0


La probabilidad empírica de padecer un tumor, también llamada **prevalencia**, es el número de casos positivos dividido por el total de sujetos o, lo que es lo mismo, la media de la columna `hadCancer`:

In [ ]:
df.hadCancer.mean()

0.123645

### Falsos positivos y falsos negativos

Supongamos que una primera prueba, la mamografía da positivo para una lesión potencialmente maligna. ¿Cuál es la probabilidad de acabar siendo diagnosticada de cáncer de mama? ¿Y si la prueba incial da negativo?

Las probabilidades de ser diagnosticada con cáncer en cada caso resultarán de restringir el data set a las condiciones correspondientes y calcular la media de `hadCancer`:

In [ ]:
df[df.mammoResult1==1].hadCancer.mean()

0.5649815237046938

In [ ]:
df[df.mammoResult1==0].hadCancer.mean()

0.019593062412337884

Fijémenos que esto es lo mismo que calcular probabilidades condiciondas:

In [ ]:
p_cancer = df.hadCancer.mean() 
#This line calculates the probability of having cancer in the entire population using the mean of the "hadCancer" column in the dataframe.
p_test_positive = df.mammoResult1.mean()
#This line calculates the probability of getting a positive result on the mammogram test using the mean of the "mammoResult1" column in the dataframe.
p_cancer_and_test_positive = df[(df.mammoResult1==1) & (df.hadCancer==1)].hadCancer.count() / len(df)
#his line calculates the probability of having cancer and getting a positive test result by counting the number of rows where both "mammoResult1" and "hadCancer" are equal to 1,
#and dividing by the total number of rows in the dataframe.

p_cancer_if_test_positive = p_cancer_and_test_positive / p_test_positive
#This is the final line of code, where it is using Bayes' theorem to calculate the probability of having cancer given a positive test result 
#by dividing the probability of both cancer and a positive test (p_cancer_and_test_positive) by the probability of a positive test (p_test_positive).


In [ ]:
print(p_cancer_if_test_positive)
print(df[df.mammoResult1==1].hadCancer.mean())

0.5649815237046937
0.5649815237046938


Podemos aprovechar los cálculos para calcular la probabilidad de obtener una prueba positiva si se padece cáncer:

In [ ]:
p_positive_if_cancer = p_cancer_and_test_positive / p_cancer
print(p_positive_if_cancer)

0.8717699866553439


Y también podemos comprobar que se obtiene el mismo resultado utilizando de **Fórmula de Bayes** de inversión de las condiciones:

In [ ]:
p_positive_if_cancer_bayes = (p_test_positive / p_cancer) * p_cancer_if_test_positive
#It is similar to the previous calculation, but using the probabilities of p(cancer|positive) and p(positive) instead of the previous ones.

print(p_positive_if_cancer)
print(p_positive_if_cancer_bayes)

0.8717699866553439
0.8717699866553438


### El sentido de una prueba que no implica diagnóstico

De estos resultados, se observa que la probabilidad de padecer una lesión maligna en presencia de una mamografía positiva es aproximadamente del 56.5%, lo cual no parece gran cosa.

Calcula la probabilidad de padecer cáncer si: 
* Salen dos positivos.
* Salen dos negativos.
* Sale al menos un negativo.

Con los resultados en la mano, razona finalmente cuál es la utilidad de la mamografía como prueba diagnóstica.

Utilicemos directamente las funcionalidades de `pandas` para calcular las probabilidades que nos interesan. Los dos primeros cálculos son directos:

In [ ]:
print(df[(df.mammoResult1==1)&(df.mammoResult2==1)].hadCancer.mean())

print(df[(df.mammoResult1==0)&(df.mammoResult2==0)].hadCancer.mean())

0.9235079171741778
0.0025994425794254815


In [ ]:
p_cancer_given_two_positives = df[(df.mammoResult1 == 1) & (df.mammoResult2 == 1) & (df.hadCancer == 1)].hadCancer.count() / df[(df.mammoResult1 == 1) & (df.mammoResult2 == 1)].hadCancer.count()
print(p_cancer_given_two_positives)

0.9235079171741778


This code uses the dataframe (df) to find the number of rows where both "mammoResult1" and "mammoResult2" are equal to 1, and "hadCancer" is also equal to 1, and then divides that number by the total number of rows where both "mammoResult1" and "mammoResult2" are equal to 1. This gives you the probability of having cancer given that two tests came back positive.


El tercer cálculo es la probabilidad de una unión, lo cual implica también calcular la probabilidad de la intersección:

In [ ]:
p_test1_positive = df.mammoResult1.mean()
p_test2_positive = df.mammoResult2.mean()
p_both_positive = df[(df.mammoResult1==1) & (df.mammoResult2==1)].Patient.count() / len(df)

print(p_test1_positive + p_test2_positive - p_both_positive)

0.28061500000000006


In [ ]:
p_cancer_given_one_negative = df[((df.mammoResult1 == 0) | (df.mammoResult2 == 0)) & (df.hadCancer == 1)].hadCancer.count() / df[(df.mammoResult1 == 0) | (df.mammoResult2 == 0)].hadCancer.count()
print(p_cancer_given_one_negative)

0.03217161164507592


Nos podemos ahorrar el uso de la fórmula de la probabilidad de la unión creando una columna auxiliar que nos diga si, por lo menos, una de las pruebas ha dado positivo:

In [ ]:
df["one_test_positive"] = (df.mammoResult1 + df.mammoResult2 > 0) * 1
df

,Patient,hadCancer,mammoResult1,mammoResult2,one_test_positive
0,1,0,0,0,0
1,2,0,0,1,1
2,3,0,0,0,0
3,4,1,1,1,1
4,5,0,0,0,0
...,...,...,...,...,...
199995,199996,0,0,0,0
199996,199997,1,1,1,1
199997,199998,0,1,0,1
199998,199999,0,0,0,0


In [ ]:
p_cancer_given_one_positive = df[((df.mammoResult1 == 1) | (df.mammoResult2 == 1)) & (df.hadCancer == 1)].hadCancer.count() / df[(df.mammoResult1 == 1) | (df.mammoResult2 == 1)].hadCancer.count()
print(p_cancer_given_one_positive)

0.4339575575076172


La media de esta nueva columna es la probabilidad de obtener al menos un test positivo:

In [ ]:
df.one_test_positive.mean()

0.280615

What we observe is that, in a series of two tests, the probability of suffering a malignant lesion if at least one of them is positive is 28%, while if both are negative, the probability drops to 0.26%. . On the other hand, it takes two positives to have a probability greater than 90%, and even then this would not provide us with a definitive diagnosis.

Generally, only one test is carried out per session, but we have seen that the probability of having a malignant lesion if a single result is negative is only 1.96%. That is, only 2 out of 100 tests are a false negative. This means that mammography is not a diagnostic test but rather a rule-out test (the diagnostic test being a biopsy of breast tissue). Its usefulness lies in screening, which on the one hand saves costs for the health system, since it is cheaper to obtain a negative result in a mammogram than in a biopsy, and also saves the patient an invasive test that requires at least outpatient hospitalization. cases where it may not be necessary.

## Ejercicio 2 - Descriptores estadísticos con datos masivos

`pandas` is very useful and efficient for calculating means, medians, variances, etc. but its effectiveness depends to a large extent on the ability we have to store our data set in memory. Such a thing is often not possible in Big Data environments and other, more refined strategies are therefore required to calculate these descriptors.

### La media por batches
Supongamos, por ejemplo, que tenemos una larga serie temporal de datos, por ejemplo la potencia suministrada por una central eléctrica, medida en Watts, con una lectura por nanosegundo (ns) a lo largo de tres meses.

En un día hay $86400\ \mathrm{s}$, y en un segundo hay $1000000\ \mathrm{ns}$, luego en un día disponemos de $8.64 \cdot 10^{10}$ lecturas. Como se trata de potencia, estas lecturas están guardadas en floats de $32\ \mathrm{bits}$, con lo cual la serie entera ocupa, por lo menos, $2.7648 \cdot 10^{12}\ \mathrm{bits}$, esto es, $3.456 \cdot 10^{11}\ \mathrm{bytes}$ o, lo que es lo mismo, casi $346\ \mathrm{Gb}$ de memoria RAM solo para guardar las lecturas de un día.

Difícilmente dispondremos de una máquina con más de 16 Gb de memoria RAM (las hay hasta de 128 Gb, por precios superiores a los 5,000 euros), así que no nos quedará más remedio que ir leyendo los datos en batches e ir acumulando los resultados para obtener el cálculo final.

Aquí no disponemos de un data set semejante, pero vamos a simular la situación con una colección de ficheros `.csv` que, aunque caben en memoria y no abarcan tres meses, leeremos secuncialmente.

Escribe una rutina que lea secuencialmente los ficheros `power_issued_XX.csv` con las lecturas de potencia de señal y que dé como resultado final la media de toda la serie.

In [ ]:
file_list = [io.BytesIO(uploaded[f'power_issued_{k + 1}.csv']) for k in np.arange(10)]

# Initialize variables to store the sum and count of readings
cumsum = 0
length = 0

# Loop through all the files in the current directory
for filename in file_list:

  df = pd.read_csv(filename, sep=",")

 # Add the sum of the "power" column to the total power
  cumsum += df.power.sum()
 # Add the number of rows to the count
  length += df.power.count()

# Calculate the final mean by dividing the total power by the count
mean_df = cumsum / length

print(mean_df)

1244474.6727138932


### La varianza
Utiliza la misma idea para calcular la varianza empírica. Da también una estimación insesgada de la varianza poblacional.

La varianza tiene la única salvedad de tener que calcular la varianza insesgada, pero no es mayor problema:

(The variance has the only caveat of having to calculate the unbiased variance, but it is not a major problem:)

In [ ]:
file_list = [io.BytesIO(uploaded[f'power_issued_{k + 1}.csv']) for k in np.arange(10)]

#  Initialize variable to store the sum of the squared deviations
cum_sq_sum = 0

for filename in file_list:

  df = pd.read_csv(filename, sep=",")
  # Iterate over the readings in the current file
  cum_sq_sum += ((mean_df - df.power)**2).sum()

var_df = np.sqrt(cum_sq_sum / length)

# Calculate the empirical variance (unbiased)
unbiased_var = var_df * ((length - 1) / length)

print(var_df, unbiased_var)

618357.200733084 618356.4850418795


# Ejercicio 3 - Probabilidades contínuas
La variable `power` puede pensarse como un muestreo de una variable aleatoria absolutamente contínua. Llamemos $X$ a la variable aleatoria que genera dichos datos. Continuando con la metodología aplicada en los apartados anteriores, calcula:
* $P(X \leq 2.5)$
* $P(1 \leq X \leq 3)$
* $P(X \geq 2.8)$
* $P(X \leq 2.5 | 1 \leq X \leq 3)$

Nota: aquí las cantidades están expresadas en megavatios (MW). 1 megavatio = 1 millón de vatios.

In [ ]:
file_list = [io.BytesIO(uploaded[f'power_issued_{k + 1}.csv']) for k in np.arange(10)]

positive_cases = 0
total_length = 0
for filename in file_list:

  df = pd.read_csv(filename, sep=",")

  positive_cases += len(df[df.power <= 1.25e06])
  total_length += len(df)

prob = positive_cases / total_length

#"positive_cases" counts the number of rows where the condition "power <= 1.25e06" is true.
#"total_length" counts the total number of rows in the dataframe.
#"prob" is calculated as the ratio of "positive_cases" to "total_length", giving the probability of the condition being true.


print("x <= 1.25", prob)

file_list = [io.BytesIO(uploaded[f'power_issued_{k + 1}.csv']) for k in np.arange(10)]

positive_cases = 0
total_length = 0
for filename in file_list:

  df = pd.read_csv(filename, sep=",")

  positive_cases += len(df[(df.power >= 1e06)&(df.power <= 3e06)])
  total_length += len(df)

prob = positive_cases / total_length

print("1 <= x <= 3", prob)

file_list = [io.BytesIO(uploaded[f'power_issued_{k + 1}.csv']) for k in np.arange(10)]

positive_cases = 0
total_length = 0
for filename in file_list:

  df = pd.read_csv(filename, sep=",")

  positive_cases += len(df[df.power >= 2.8e06])
  total_length += len(df)

prob = positive_cases / total_length

print("x >= 2.8", prob)

file_list = [io.BytesIO(uploaded[f'power_issued_{k + 1}.csv']) for k in np.arange(10)]

positive_cases = 0
total_length = 0
for filename in file_list:

  df = pd.read_csv(filename, sep=",")

  df = df[(df.power >= 1e06)&(df.power <= 3e06)]

  positive_cases += len(df[df.power <= 1.25e06])
  total_length += len(df)

prob = positive_cases / total_length

#The dataframe is filtered to only include rows where the condition "1e06 <= power <= 3e06" is true.
#"positive_cases" counts the number of rows where the condition "power <= 1.25e06" is true in the filtered dataframe.
#"total_length" counts the total number of rows in the filtered dataframe.
#"prob" is calculated as the ratio of "positive_cases" to "total_length", giving the probability of the condition being true, given that "1e06 <= power <= 3e06" is true.

print("x <= 1.25 condicionada a 1 <= x <= 3", prob)

x <= 1.25 0.44285416666666666
1 <= x <= 3 0.6717418981481481
x >= 2.8 5.3240740740740744e-05
x <= 1.25 condicionada a 1 <= x <= 3 0.17059882664093662


# Problema - ¿Quién ganará las elecciones?

Tres partidos se presentan a las elecciones presidenciales de un cierto país. Para conocer el estado de opinión de la población y las expectitivas de cada uno, el gobierno encarga una encuesta electoral a una empresa de sociométrica justo antes de la campaña electoral. También pretenden usar el estudio para conocer quién hace mejor campaña electoral.

Carga el fichero `elections.csv` con los resultados de la encuesta. Trataremos de responder a las dos cuestiones.

In [ ]:
import io
import numpy as np
import pandas as pd

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving elections.csv to elections.csv


In [ ]:
path = io.BytesIO(uploaded['elections.csv'])
df = pd.read_csv(path, sep=",")
df
#Hay que tener en cuenta que en la columna gender mujer=1 y hombre=0

,Unnamed: 0,votes_for_A,votes_for_B,votes_for_C,gender,p_changes_vote
0,1,0,1,0,1,0.062710
1,2,0,0,1,0,0.409047
2,3,0,0,1,1,0.210734
3,4,0,0,1,1,0.304020
4,5,1,0,0,0,0.317170
...,...,...,...,...,...,...
3495,3496,1,0,0,0,0.307702
3496,3497,0,0,1,1,0.438484
3497,3498,0,0,1,1,0.034694
3498,3499,0,0,0,0,0.000000


In [ ]:
df.dtypes

Unnamed: 0          int64
votes_for_A         int64
votes_for_B         int64
votes_for_C         int64
gender              int64
p_changes_vote    float64
dtype: object

## Parte 1: Interpretación directa de la encuesta

Basándote únicamente en lo declarado por los entrevistados, calcula las probabilidades de voto de cada uno de los partidos. Para cada partido, calcula la probabilidad de voto si se es mujer o si se es hombre. Tomando un voto del partido A al azar, ¿cuál es la probabilidad de que provenga de una mujer? ¿Y si el voto es del partido C?

In [ ]:
#probabilidades de voto de cada uno de los partidos

prob_party_a=round(df.votes_for_A.mean()*100,2)
prob_party_b=round(df.votes_for_B.mean()*100,2)
prob_party_c=round(df.votes_for_C.mean()*100,2)
print('PROBABILIDAD DE VOTOS % PARA EL PARTIDO A:',prob_party_a,'%')
print('PROBABILIDAD DE VOTOS % PARA EL PARTIDO B:',prob_party_b,'%')
print('PROBABILIDAD DE VOTOS % PARA EL PARTIDO C:',prob_party_c,'%')

PROBABILIDAD DE VOTOS % PARA EL PARTIDO A: 39.8 %
PROBABILIDAD DE VOTOS % PARA EL PARTIDO B: 15.54 %
PROBABILIDAD DE VOTOS % PARA EL PARTIDO C: 25.17 %


**Para cada partido, calcula la probabilidad de voto si se es mujer o si se es hombre.**

In [ ]:


# Probabilidad de voto siendo mujer
print("La probabilidad de haber votado al partido A siendo mujer es de", round(df[df.gender==1].votes_for_A.mean()*100,2),"%")
print("La probabilidad de haber votado al partido B siendo mujer es de", round(df[df.gender==1].votes_for_B.mean()*100,2),"%")
print("La probabilidad de haber votado al partido C siendo mujer es de", round(df[df.gender==1].votes_for_C.mean()*100,2),"%")



La probabilidad de haber votado al partido A siendo mujer es de 39.65 %
La probabilidad de haber votado al partido B siendo mujer es de 18.33 %
La probabilidad de haber votado al partido C siendo mujer es de 42.02 %


In [ ]:
# Probabilidad de voto siendo hombre
print("La probabilidad de haber votado al partido A siendo hombre es de", round(df[df.gender==0].votes_for_A.mean()*100,2),"%")
print("La probabilidad de haber votado al partido B siendo hombre es de", round(df[df.gender==0].votes_for_B.mean()*100,2),"%")
print("La probabilidad de haber votado al partido C siendo hombre es de", round(df[df.gender==0].votes_for_C.mean()*100,2),"%")

La probabilidad de haber votado al partido A siendo hombre es de 39.91 %
La probabilidad de haber votado al partido B siendo hombre es de 13.52 %
La probabilidad de haber votado al partido C siendo hombre es de 12.93 %


**Tomando un voto del partido A al azar, ¿cuál es la probabilidad de que provenga de una mujer? ¿Y si el voto es del partido C?**

In [ ]:
# Probabilidad de que tomando un voto al azar del partido A, sea mujer
# Hacemos probabilidad condicionada: P(A|B)= P(A∩B)/P(B)
# En este caso A='Es mujer' y B='Voto para el partido A'

dfA_f=df[df.votes_for_A==1]
print("La probabilidad de que tomando un voto al azar del partido A, este sea de una mujer es de", round(len(dfA_f[dfA_f.gender==1])/len(dfA_f)*100,2),"%")

# Repetimos la idea para los votos del partido C

dfC_f=df[df.votes_for_C==1]
print("La probabilidad de que tomando un voto al azar del partido C, este sea de una mujer es de", round(len(dfC_f[dfC_f.gender==1])/len(dfC_f)*100,2),"%")

La probabilidad de que tomando un voto al azar del partido A, este sea de una mujer es de 41.92 %
La probabilidad de que tomando un voto al azar del partido C, este sea de una mujer es de 70.26 %


## Parte 2: Volatilidad del voto

La columna `p_changes_vote` muestra la probabilidad de que el encuestado termine votando algo distinto a lo declarado. Úsala para determinar la fiabilidad de la encuesta. ¿Es igual de volátil el voto femenino que el masculino?

In [ ]:
# Probabilidad de cambio de voto si es mujer o si es hombre

mean_p_changes_vote_women = df[df["gender"] == "1"]["p_changes_vote"].mean()
print("La probabilidad de canviar de voto siendo mujer es",round(df[df["gender"] == 1]["p_changes_vote"].mean()*100,2),"%")
print("La probabilidad de canviar de voto siendo hombre es",round(df[df["gender"] == 0]["p_changes_vote"].mean()*100,2),"%")

La probabilidad de canviar de voto siendo mujer es 24.03 %
La probabilidad de canviar de voto siendo hombre es 17.69 %


Viendo los datos, puede parecer que la volatilidad en el voto femenino es mayor al masculino. 

## Parte 3: Análisis de campaña

**(a)** Suponiendo que los resultados finales son 

* partido A -> 38.77%,
* partido B -> 15.04%,
* partido C -> 29.16%,

calcula la probabilidad de que, tomado un voto del partido A, éste provenga de un entrevistado que declaró querer votar al partido A. Haz lo mismo con los partidos B y C.

X -> Vota a A 

Y -> Declara votar a A

P(Y|X) = P(Y∩X)/P(X) = P(Y)P(Y∩X)/(P(Y)P(X))=P(X|Y)P(Y)/P(X)

In [ ]:
#Final votes
p_final_voto_A = 0.3877
p_final_voto_B = 0.1504
p_final_voto_C = 0.2916

In [ ]:
#Estimated votes
p_dijo_vote_A = df.votes_for_A.mean()
p_dijo_vote_B =df.votes_for_B.mean()
p_dijo_vote_C = df.votes_for_C.mean()

In [ ]:

# Partido A
p_vote_A_if_dijo_A = 1-df[df['votes_for_A'] == 1].p_changes_vote.mean()
p_vote_A_if_dijo_A

p_say_A_if_vote_A = (p_vote_A_if_dijo_A * p_dijo_vote_A) / p_final_voto_A
print("La probabilidad de que tomado un voto del partido A, éste provenga de un entrevistado que declaró querer votar al partido A es de", round(p_say_A_if_vote_A,2)*100, "%")
 



La probabilidad de que tomado un voto del partido A, éste provenga de un entrevistado que declaró querer votar al partido A es de 66.0 %


In [ ]:
# Partido B
p_vote_B_if_dijo_B = 1-df[df['votes_for_B'] == 1].p_changes_vote.mean()
p_vote_B_if_dijo_B

p_say_B_if_vote_B = (p_vote_B_if_dijo_B * p_dijo_vote_B) / p_final_voto_B
print("La probabilidad de que tomado un voto del partido B, éste provenga de un entrevistado que declaró querer votar al partido B es de", round(p_say_B_if_vote_B,2)*100, "%")
 

La probabilidad de que tomado un voto del partido B, éste provenga de un entrevistado que declaró querer votar al partido B es de 98.0 %


In [ ]:
# Partido C
p_vote_C_if_dijo_C = 1-df[df['votes_for_C'] == 1].p_changes_vote.mean()
p_vote_C_if_dijo_C

p_say_C_if_vote_C = (p_vote_C_if_dijo_C * p_dijo_vote_C) / p_final_voto_C
print("La probabilidad de que tomado un voto del partido c, éste provenga de un entrevistado que declaró querer votar al partido C es de", round(p_say_C_if_vote_C,2)*100, "%")
 

La probabilidad de que tomado un voto del partido c, éste provenga de un entrevistado que declaró querer votar al partido C es de 68.0 %


**(b)** Suponiendo que el estado reparte 10 millones de euros en subvenciones a cada partido en función de sus resultados electorales y que cada partido invirtió en la campaña las cantidades siguientes

* partido A -> 3.8 millones,
* partido B -> 1.47 millones,
* partido C -> 2.31 millones,

calcula cuánto esperaban ganar con las subvenciones y termina deduciendo quién hizo mejor campaña electoral.

In [ ]:
inversion_partido_A = 3800000
inversion_partido_B = 1470000 
inversion_partido_C = 2310000

subvencion = 10000000

ganancia_A = subvencion - inversion_partido_A
ganancia_B = subvencion - inversion_partido_B
ganancia_C = subvencion - inversion_partido_C

print("En toda la campaña, cada partido estimaba ganar como máximo el siguiente importe:")
print("A:", ganancia_A)
print("B:", ganancia_B)
print("C:", ganancia_C)

En toda la campaña, cada partido estimaba ganar como máximo el siguiente importe:
A: 6200000
B: 8530000
C: 7690000


In [ ]:
# Entendemos que el "éxito" de la campaña se enfoca en los indecisos.
# ÉxitoParido = Definitivos - Votaria

exito_A = p_final_voto_A - df.votes_for_A.mean()
exito_B = p_final_voto_B - df.votes_for_B.mean()
exito_C = p_final_voto_C - df.votes_for_C.mean()

print("En toda la campaña, cada partido ganó o perdió el siguiente % de votos")
print("A:", round(exito_A*100,2), "%")
print("B:", round(exito_B*100,2), "%")
print("C:", round(exito_C*100,2), "%")

En toda la campaña, cada partido ganó o perdió el siguiente % de votos
A: -1.03 %
B: -0.5 %
C: 3.99 %


Podemos ver que el partido C gano los mas votos de indecisos.

In [ ]:

# party vote percentages
expected_party_votes={'A': 49.43, 'B': 19.30, 'C': 31.26}

party_votes = {'A': 38.77, 'B': 15.04, 'C': 29.16}

# party campaign investments
party_investments = {'A': 3.8, 'B': 1.47, 'C': 2.31}

# subsidies per party
subsidies = 10

# expected to earn in total
hoped_to_earn = {party: subsidies * (vote/100) for party, vote in expected_party_votes.items()}

# estimated profit based on first estimation

expected_profit = {party: hoped_to_earn[party] - party_investments[party] for party in expected_party_votes.keys()}

# calculate hoped to earn for each party
earned = {party: subsidies * (vote/100) for party, vote in party_votes.items()}

# calculate profit for each party
profit = {party: earned[party] - party_investments[party] for party in party_votes.keys()}

# display results
print('Hoped to earn:')
print(hoped_to_earn)
print('Actual earnings:')
print(earned)
print('Actual profit:')
print(profit)

Hoped to earn:
{'A': 4.9430000000000005, 'B': 1.9300000000000002, 'C': 3.126}
Actual earnings:
{'A': 3.8770000000000007, 'B': 1.5039999999999998, 'C': 2.9160000000000004}
Actual profit:
{'A': 0.07700000000000085, 'B': 0.03399999999999981, 'C': 0.6060000000000003}


In [ ]:
best_party = max(profit, key=profit.get)
print(f"Partido {best_party} hizo la mejor campana ")

Partido C hizo la mejor campana


Porque? Tenemos en cuenta cuánto gastaron y cuál fue el beneficio real monetario para ellos.